In [4]:
from sqlalchemy import text
from app import engine
import time
import pandas as pd


In [ ]:
database = []
query_plan_log = "query_plan_log.txt"
# Example in Python using a decorator
def log_query_time(func):
    def wrapper(*args,**kwargs):
        start = time.perf_counter()
        result = func(args[0], args[1], description = kwargs['description'])
        duration = time.perf_counter() - start

        query_duration = {'query':args[0], 'duration':duration, 'Description':kwargs['description']}
        database.append(query_duration)
        with open(query_plan_log,'a') as f:
            f.write(f'{query_duration}\n')
        print(query_duration)
        return result
    return wrapper

@log_query_time
def execute_query(query,conn,description=""):
    # Database execution logic
    df = pd.read_sql_query(query,con=conn)
    #return df

In [ ]:
candidate_index = ['Category_id','Content_Rating','Price','Rating']

In [90]:
idxname = "price,rating,content_rating,category_id" #Example
sql_create_index = text(f"CREATE INDEX IF NOT EXISTS idx_apps_{idxname.replace(",","_").lower()} ON apps({idxname});")
sql_drop_index = text(f"DROP INDEX IF EXISTS idx_apps_{idxname.replace(",","_")};")

with engine.connect() as conn:
    #Drop Existing Indexes
    #Query Indexes all existing indexes
    query_index = "SELECT INDEXNAME, TABLENAME FROM PG_INDEXES\
    WHERE SCHEMANAME = 'googleplay' ORDER BY INDEXNAME"
    indexes = pd.read_sql_query(query_index,con=conn)
    for idxname in indexes.indexname:
        if idxname.startswith('idx'):
            conn.execute(text(f"DROP INDEX {idxname};"))
        conn.commit()

    #First Set index 
    try:
        conn.execute(sql_create_index)
        conn.commit()
    except Exception as e:
        print(f"Failed to create index: {e}")

    query = "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id\
    WHERE price between 1 AND 10 AND category='Education' AND rating >= 2.5 ;"

    execute_query(query,conn,description=f"Using Index on {idxname}")

    #Drop Index
    conn.execute(sql_drop_index)
    conn.commit()
    execute_query(query,conn,description=f"Not Using Index For {idxname}")

{'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id    WHERE price between 1 AND 10 AND category='Education' AND rating >= 2.5 ;", 'duration': 0.05136730000958778, 'Description': 'Using Index on idx_apps_price_rating_content_rating_category_id'}
{'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id    WHERE price between 1 AND 10 AND category='Education' AND rating >= 2.5 ;", 'duration': 0.37495769999804907, 'Description': 'Not Using Index For idx_apps_price_rating_content_rating_category_id'}


In [78]:
#After Experimenting multiple indexes show the best result
logs = sorted(database,key=lambda x:x['duration'])

In [81]:
logs[:10]

[{'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id    WHERE rating > 3.5 AND price >= 4.99  AND category='Entertainment' AND content_rating='Teen';",
  'duration': 0.004771700012497604,
  'Description': 'Using Index on price,rating,content_rating,category_id'},
 {'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id    WHERE rating > 3.5 AND price >= 4.99  AND category='Entertainment' AND content_rating='Teen';",
  'duration': 0.005151799996383488,
  'Description': 'Using Index on price,rating,category_id'},
 {'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.category_id    WHERE rating > 3.5 AND price >= 4.99  AND category='Entertainment' AND content_rating='Teen';",
  'duration': 0.005164400005014613,
  'Description': 'Using Index on price,rating,category_id,content_rating'},
 {'query': "SELECT * FROM APPS  INNER JOIN Categories on apps.category_id=Categories.catego